# Generieren der Bilder:

Beim generieren der Bilder wurde sich am Angfang auf Bilder von Autos beschränkt, da eine bestimmte "Kategorie" an Bilder den Erkennungsprozess erheblich erleichtern.

Um ein möglichst ausbalanciertes Dataset an KI-generierten Bilder zu erlangen, werden verschiedene prompts genutzt: \
    1) "familycar" \
    2) "sportscar" \
    3) "old car"

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda"

pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to(device)

def generateImages(amountOfEachPrompt, prompt, variations, targetdir):
    for i in range (amountOfEachPrompt):
        for i2 in variations:
            newprompt = i2 + prompt
            image = pipe(i2 + prompt).images[0]
            image.save(targetdir + newprompt.replace(' ', '_') + str(i) + ".png")

In [ ]:
prompt = "car"
variations = ["family", "sports", "old "]
amountOfEachPrompt = 20

targetDir = "./"

generateImages(amountOfEachPrompt, prompt, variations, targetDir)

## Fazit der ersten Generierung:

 + Resulate waren gut. Bilder konnten überraschend gut klassifiziert werden.
 
 - Bilder mit dem prompt familycar beinhalteten oft Menschen, welche deutlich erkennbar generiert sind. Im Dataset der realen Bilder befinden sich deutlich weniger menschen auf den Fotos.
 - Stable Diffusion 1.4 ist etwas veraltet --> neuere Modelle, welche realistischere Bilder erzeugen


## Reale Bilder


Link: https://www.kaggle.com/datasets/jessicali9530/stanford-cars-dataset/data

In dem Dataset wird angegeben, dass die Bilder labels beinhalten: Baujahr, Modellbezeichnung, ...

In [2]:
from scipy.io import loadmat

# Pfad zur .mat Datei
mat_file_path = './images/car/cars_annos.mat'

# Laden der .mat Datei
mat_contents = loadmat(mat_file_path)

In [ ]:
Labels passen aus meiner Sicht nicht zu den Bildern --> Deshalb eigenes Feature-Extracting

In [27]:
from scipy.io import loadmat

# Laden der .mat Datei
mat_file = './images/car/cars_annos.mat'
mat_contents = loadmat(mat_file)

# Extrahieren der Annotations und Class Names
annotations = mat_contents['annotations']
class_names = mat_contents['class_names'][0]

# Erstellen eines Dictionaries, das die Bildpfade auf ihre Labels abbildet
image_label_map = {}

for annotation in annotations[0]:
    image_path = annotation[0][0].split('/')[1]  # Pfad zum Bild
    class_index = annotation[5][0][0] - 1  # Index des Labels (angepasst für 0-basierte Indizierung)
    label = class_names[class_index][0]  # Das tatsächliche Label
    image_label_map[image_path] = label
